In [ ]:
import lightning.pytorch as pl

from src.lightning_classes import UnrolledSystem
from src.data_loader import RGBDataset
from src.utils import get_dataloader

In [ ]:
CFAS_TRAIN = ['bayer_GRBG', 'binning', 'chakrabarti', 'gindele', 'hamilton', 'honda', 'kaizu', 'kodak', 'quad_bayer', 'random', 'sparse_3', 'wang', 'yamagami', 'yamanaka']
CFAS_TEST = ['honda2', 'lukac', 'luo', 'sony', 'xtrans']
TEST_DIR = 'input/test'
BATCH_SIZE = 16

In [ ]:
dataset = RGBDataset(TEST_DIR, CFAS_TRAIN, cfa_variants=0)
dataloader = get_dataloader(dataset, BATCH_SIZE)

dataset_test = RGBDataset(TEST_DIR, CFAS_TEST, cfa_variants=0)
dataloader_test = get_dataloader(dataset_test, BATCH_SIZE)

experiment = 'bayer_GRBG-binning-chakrabarti-gindele-hamilton-honda-kaizu-kodak-quad_bayer-sparse_3-wang-yamagami-yamanaka-6V'
version = 'version_0'
model = UnrolledSystem.load_from_checkpoint(f'weights/{experiment}/{version}/checkpoints/best.ckpt')

trainer = pl.Trainer(logger=False)

In [ ]:
trainer.test(model=model, dataloaders=[dataloader, dataloader_test]);